In [1]:
# Configure warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", 
                      message=".*only valid for n>=20.*continuing anyway.*",
                      category=UserWarning,
                      module="scipy.stats.*")
warnings.filterwarnings("ignore", 
                      message="The behavior of wald_test will change after 0.14.*",
                      category=FutureWarning,
                      module="statsmodels.*")

# Stepwise Selection Examples (OLS and GLM)
This notebook demonstrates stepwise selection with AIC/BIC/AdjR2/p-value, including GLM test options, interactions, and transformed terms.

In [2]:
# Imports
import pandas as pd
import statsmodels.api as sm
from step_criterion import step_criterion, step_aic, step_bic, step_adjr2, step_pvalue

## Load public datasets
We'll use statsmodels built-in datasets:
- OLS: Longley dataset (economic data, continuous response)
- GLM (Binomial): Heart disease dataset (binary response)

In [3]:
# Longley (OLS)
longley = sm.datasets.longley.load_pandas().data
longley.rename(columns={'TOTEMP':'y'}, inplace=True)  # response y
longley.head()

,y,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,60323.0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,61122.0,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,60171.0,88.2,258054.0,3682.0,1616.0,109773.0,1949.0
3,61187.0,89.5,284599.0,3351.0,1650.0,110929.0,1950.0
4,63221.0,96.2,328975.0,2099.0,3099.0,112075.0,1951.0


In [4]:
# South African Heart Disease (GLM Binomial)
heart = sm.datasets.ccard.load_pandas().data  # Credit card dataset as fallback
# Let's use a different dataset with more predictors
try:
    # Try to load a dataset with more variables
    import numpy as np
    np.random.seed(42)
    n = 200
    heart = pd.DataFrame({
        'age': np.random.normal(50, 10, n),
        'tobacco': np.random.exponential(5, n),
        'ldl': np.random.normal(4, 1.5, n),
        'adiposity': np.random.normal(25, 5, n),
        'famhist': np.random.choice([0, 1], n, p=[0.7, 0.3]),
        'typea': np.random.normal(50, 15, n),
        'obesity': np.random.normal(25, 3, n),
        'alcohol': np.random.exponential(10, n)
    })
    # Create target with some relationship to predictors
    logit = -5 + 0.05*heart['age'] + 0.1*heart['tobacco'] + 0.3*heart['ldl'] + heart['famhist']
    heart['target'] = (np.random.random(n) < 1/(1+np.exp(-logit))).astype(int)
except:
    heart = sm.datasets.ccard.load_pandas().data
    heart['target'] = heart.iloc[:, -1].astype(int)
    
heart.head()

,age,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,target
0,54.967142,10.493043,3.836860,29.355624,0,53.740755,27.610203,9.320465,0
1,48.617357,6.750171,4.602568,23.369882,0,73.661799,26.487046,14.473607,0
2,56.476885,5.970372,5.035216,31.006070,0,48.570567,25.451257,1.921763,1
3,65.230299,6.061438,3.398169,22.959623,1,54.185323,26.094883,6.982925,1
4,47.658466,2.227462,4.336139,14.809377,0,59.118448,32.210247,5.085998,0


## Example 1: OLS with BIC and interactions/squared terms

In [5]:
df = longley.copy()
initial = 'y ~ 1'
upper = 'y ~ GNP + UNEMP + ARMED + POP + YEAR + GNPDEFL + GNP:YEAR + I(GNP**2)'
res_bic = step_criterion(data=df, initial=initial, scope={'upper': upper}, direction='both', criterion='bic', trace=1)
res_bic.model.summary()

Start:  AIC=308.392
y ~ 1 

Op      Term        BIC        AIC
 +       GNP 256.402196 256.402196
 +  GNP:YEAR 256.573160 256.573160
 +      YEAR 265.193585 265.193585
 + I(GNP**2) 265.388583 265.388583
 +   GNPDEFL 265.428786 265.428786
 +       POP 270.275706 270.275706
 +     UNEMP 306.507988 306.507988
 +     ARMED 307.410468 307.410468

Step:  AIC=256.402
y ~ 1 + GNP 

Op      Term        BIC        AIC
 +     UNEMP 250.812174 250.812174
 +       POP 252.080642 252.080642
 +  GNP:YEAR 255.808838 255.808838
 +      YEAR 255.874007 255.874007
 + I(GNP**2) 257.785152 257.785152
 +   GNPDEFL 258.602882 258.602882
 +     ARMED 258.970301 258.970301
 -       GNP 308.391828 308.391828

Step:  AIC=250.812
y ~ 1 + GNP + UNEMP 

Op      Term        BIC        AIC
 +     ARMED 249.407754 249.407754
 +      YEAR 251.988693 251.988693
 +       POP 253.145959 253.145959
 + I(GNP**2) 253.490888 253.490888
 +   GNPDEFL 253.500313 253.500313
 +  GNP:YEAR 253.584479 253.584479
 -     UNEMP 256.4021

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     589.8
Date:                Wed, 20 Aug 2025   Prob (F-statistic):           9.50e-13
Time:                        18:57:05   Log-Likelihood:                -109.83
No. Observations:                  16   AIC:                             229.7
Df Residuals:                      11   BIC:                             233.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.599e+06   7.41e+05     -4.859      0.001   -5.23e+06   -1.97e+06
GNP           -0.0402      0.016     -2.440      0.033      -0.076      -0.004
UNEMP         -2.0884      0.290     -7.202      0.000      -2.727      -1.450
ARMED         -1.0146      0.184     -5.522      0.000      -1.419      -0.610
YEAR        1887.4095    382.766      4.931      0.000    1044.946    2729.873
==============================================================================
Omnibus:                        0.469   Durbin-Watson:                   2.604
Prob(Omnibus):                  0.791   Jarque-Bera (JB):                0.469
Skew:                           0.332   Prob(JB):                        0.791
Kurtosis:                       2.489   Cond. No.                     4.24e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.24e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# Show step path
res_bic.anova

,Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
0,,NaN,NaN,15,305.619239,308.391828
1,+ GNP,1.0,54.762220,14,250.857019,256.402196
2,+ UNEMP,1.0,8.362611,13,242.494408,250.812174
3,+ ARMED,1.0,4.177009,12,238.317399,249.407754
4,+ YEAR,1.0,18.662353,11,219.655047,233.517990


## Example 2: OLS with Adjusted R^2

In [7]:
res_adj = step_adjr2(data=df, initial='y ~ GNP + UNEMP + ARMED', scope={'upper': upper}, direction='both', trace=0)
res_adj.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     589.8
Date:                Wed, 20 Aug 2025   Prob (F-statistic):           9.50e-13
Time:                        18:57:05   Log-Likelihood:                -109.83
No. Observations:                  16   AIC:                             229.7
Df Residuals:                      11   BIC:                             233.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.599e+06   7.41e+05     -4.859      0.001   -5.23e+06   -1.97e+06
GNP           -0.0402      0.016     -2.440      0.033      -0.076      -0.004
UNEMP         -2.0884      0.290     -7.202      0.000      -2.727      -1.450
ARMED         -1.0146      0.184     -5.522      0.000      -1.419      -0.610
YEAR        1887.4095    382.766      4.931      0.000    1044.946    2729.873
==============================================================================
Omnibus:                        0.469   Durbin-Watson:                   2.604
Prob(Omnibus):                  0.791   Jarque-Bera (JB):                0.469
Skew:                           0.332   Prob(JB):                        0.791
Kurtosis:                       2.489   Cond. No.                     4.24e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.24e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Example 3: GLM Binomial with LR and Wald tests

In [8]:
dfg = heart.copy()
initial_g = 'target ~ 1'
upper_g = 'target ~ age + tobacco + ldl + adiposity + famhist + typea + obesity + alcohol + age:ldl + I(age**2)'
res_lr = step_criterion(data=dfg, initial=initial_g, scope={'upper': upper_g}, direction='both', criterion='p-value', family=sm.families.Binomial(), glm_test='lr', trace=1)
print(res_lr.model.summary())

Start:  AIC=271.205
target ~ 1 

Op      Term  p-value        AIC
 +   famhist 0.001434 263.043048
 + I(age**2) 0.006604 265.826914
 +       age 0.012193 266.922123
 +   age:ldl 0.023437 268.068830
 +   tobacco 0.067372 269.858757
 +       ldl 0.197589 271.544557
 +     typea 0.444835 272.620888
 +   obesity 0.664253 273.016268
 + adiposity 0.767492 273.117254
 +   alcohol 0.959608 273.202102

Step:  AIC=263.043
target ~ 1 + famhist 

Op      Term  p-value        AIC
 -   famhist 0.001434 271.204667
 + I(age**2) 0.004977 257.155166
 +       age 0.007854 257.976676
 +   age:ldl 0.011471 258.652229
 +   tobacco 0.077560 261.927752
 +       ldl 0.167011 263.133476
 +     typea 0.488472 264.563157
 +   obesity 0.589953 264.752630
 +   alcohol 0.922352 265.033547
 + adiposity 0.959674 265.040491

Step:  AIC=257.155
target ~ 1 + famhist + I(age**2) 

Op      Term  p-value        AIC
 -   famhist 0.001088 265.826914
 - I(age**2) 0.004977 263.043048
 +   tobacco 0.048098 255.248605
 +   age:ld

In [9]:
# Wald test variant
res_wald = step_pvalue(data=dfg, initial=initial_g, scope={'upper': upper_g}, family=sm.families.Binomial(), glm_test='wald', trace=1)

print(res_wald.anova)
print(res_wald.model.summary())

Start:  AIC=271.205
target ~ 1 

Op      Term  p-value        AIC
 +   famhist 0.001582 263.043048
 +       age 0.014225 266.922123
 +   age:ldl 0.025942 268.068830
 +   tobacco 0.070152 269.858757
 +       ldl 0.199663 271.544557
 +     typea 0.445366 272.620888
 +   obesity 0.664552 273.016268
 + adiposity 0.767576 273.117254
 +   alcohol 0.959591 273.202102
 + I(age**2)      NaN 265.826914

Step:  AIC=263.043
target ~ 1 + famhist 

Op      Term  p-value        AIC
 -   famhist 0.001582        NaN
 +       age 0.009726 257.976676
 +   age:ldl 0.013337 258.652229
 +   tobacco 0.082273 261.927752
 +       ldl 0.169319 263.133476
 +     typea 0.488507 264.563157
 +   obesity 0.590542 264.752630
 +   alcohol 0.922271 265.033547
 + adiposity 0.959673 265.040491
 + I(age**2)      NaN 257.155166

Step:  AIC=257.977
target ~ 1 + famhist + age 

Op      Term  p-value        AIC
 -   famhist 0.001095        NaN
 -       age 0.009726        NaN
 +   tobacco 0.052035 256.081842
 +   age:ldl 0.09

### Notes on glm_test='score' and 'gradient'
- These options currently fall back to LR testing with a warning.
- If you need true score tests, open an issue; we can explore adding a formal score test path.

### Warning Handling
- The package automatically suppresses common warnings from scipy.stats (small sample size warnings) and statsmodels (future behavior warnings for Wald tests)
- This provides a cleaner user experience while maintaining underlying functionality
- Important warnings that affect results are still displayed

## Model Averaging

Model averaging provides AIC/BIC weights for each model in the stepwise path, allowing you to assess the relative support for different models and compute model-averaged estimates.

The weights are calculated as:
- Δ_i = criterion_i - min(criterion)  
- w_i = exp(-0.5 * Δ_i) / Σ exp(-0.5 * Δ_j)

Where models with lower AIC/BIC values receive higher weights.

In [10]:
# Basic model averaging with AIC
result_aic_avg = step_aic(longley, initial='y ~ 1', 
                         scope='y ~ GNPDEFL + GNP + UNEMP + ARMED + POP + YEAR', 
                         model_averaging=True, trace=1)

print("Final model:")
print(result_aic_avg.model.summary().tables[1])
print("\nModel weights:")
print(result_aic_avg.model_weights)

Start:  AIC=307.619
y ~ 1 

Op    Term  Score (AIC)        AIC
 +     GNP   254.857019 254.857019
 +    YEAR   263.648407 263.648407
 + GNPDEFL   263.883609 263.883609
 +     POP   268.730529 268.730529
 +   UNEMP   304.962811 304.962811
 +   ARMED   305.865291 305.865291

Step:  AIC=254.857
y ~ 1 + GNP 

Op    Term  Score (AIC)        AIC
 +   UNEMP   248.494408 248.494408
 +     POP   249.762876 249.762876
 +    YEAR   253.556241 253.556241
 + GNPDEFL   256.285116 256.285116
 +   ARMED   256.652535 256.652535
 -     GNP   307.619239 307.619239

Step:  AIC=248.494
y ~ 1 + GNP + UNEMP 

Op    Term  Score (AIC)        AIC
 +   ARMED   246.317399 246.317399
 +    YEAR   248.898339 248.898339
 +     POP   250.055604 250.055604
 + GNPDEFL   250.409958 250.409958
 -   UNEMP   254.857019 254.857019
 -     GNP   304.962811 304.962811

Step:  AIC=246.317
y ~ 1 + GNP + UNEMP + ARMED 

Op    Term  Score (AIC)        AIC
 +    YEAR   229.655047 229.655047
 +     POP   245.876033 245.876033
 + GNP

In [11]:
# BIC model averaging (more conservative)
result_bic_avg = step_bic(longley, initial='y ~ 1', 
                         scope='y ~ GNPDEFL + GNP + UNEMP + ARMED + POP + YEAR', 
                         model_averaging=True, trace=1)

print("BIC Model weights:")
print(result_bic_avg.model_weights)
print(f"\nNumber of models with substantial support (weight > 0.1): {(result_bic_avg.model_weights['Weight'] > 0.1).sum()}")
print(f"Cumulative weight of top 2 models: {result_bic_avg.model_weights['Weight'].head(2).sum():.3f}")

Start:  AIC=308.392
y ~ 1 

Op    Term        BIC        AIC
 +     GNP 256.402196 256.402196
 +    YEAR 265.193585 265.193585
 + GNPDEFL 265.428786 265.428786
 +     POP 270.275706 270.275706
 +   UNEMP 306.507988 306.507988
 +   ARMED 307.410468 307.410468

Step:  AIC=256.402
y ~ 1 + GNP 

Op    Term        BIC        AIC
 +   UNEMP 250.812174 250.812174
 +     POP 252.080642 252.080642
 +    YEAR 255.874007 255.874007
 + GNPDEFL 258.602882 258.602882
 +   ARMED 258.970301 258.970301
 -     GNP 308.391828 308.391828

Step:  AIC=250.812
y ~ 1 + GNP + UNEMP 

Op    Term        BIC        AIC
 +   ARMED 249.407754 249.407754
 +    YEAR 251.988693 251.988693
 +     POP 253.145959 253.145959
 + GNPDEFL 253.500313 253.500313
 -   UNEMP 256.402196 256.402196
 -     GNP 306.507988 306.507988

Step:  AIC=249.408
y ~ 1 + GNP + UNEMP + ARMED 

Op    Term        BIC        AIC
 +    YEAR 233.517990 233.517990
 +     POP 249.738976 249.738976
 -   ARMED 250.812174 250.812174
 + GNPDEFL 251.751626

### Interpreting Model Weights

- **Weight > 0.1**: Models with substantial support
- **Weight > 0.05**: Models with some support  
- **Cumulative weight**: How much support the top k models have together
- **BIC vs AIC**: BIC tends to favor simpler models (fewer variables) than AIC

Model averaging is particularly useful when:
1. Multiple models have similar performance
2. You want to account for model uncertainty
3. Making predictions across different model structures

⚠️ **Important**: Model averaging weights reflect relative support among the models in your stepwise path, not all possible models. The stepwise path depends on the starting model and search direction.